In [1]:
import numpy as np
import torch
import pdb
import torch.nn as nn
import random
import matplotlib.pyplot as plt
import struct
import gzip
from PIL import Image
def load_data_image():
    #读取文件
    binfile = open('../data/train-images.idx3-ubyte', 'rb')
    buffers = binfile.read()
    #/读取前四个无符号整数
    magic, num, cows, cols = struct.unpack_from('>IIII',buffers,0)
    #获得偏移
    offset = struct.calcsize('>IIII')
    #计算所读取的数据大小num*cows*cols个bit，>表示大端读取
    strString = '>' + str(num*cows*cols) + 'B'
    #读取图片数据并转化矩阵
    training_image = struct.unpack_from(strString, buffers, offset)
    training_image = np.reshape(training_image,(num,cows*cols))
    binfile.close()
    return (training_image)
def load_data_lable():
    #读取文件
    binfile = open('../data/train-labels.idx1-ubyte', 'rb')
    buffers = binfile.read()
    #/读取前两个无符号整数
    magic, num = struct.unpack_from('>II',buffers,0)
    #获得偏移
    offset = struct.calcsize('>II')
    #计算所读取的数据大小num*cows*cols个bit，>表示大端读取
    strString = '>' + str(num) + 'B'
    #读取图片数据并转化矩阵
    training_lable = struct.unpack_from(strString, buffers, offset)
    training_lable = np.reshape(training_lable,(num))
    binfile.close()
    return (training_lable)
def I(z):#S型神经元
    for x in range(784):
        z[x] = z[x]>0
    return z
training_image = load_data_image()
training_lable = load_data_lable()
training_lables = np.zeros((60000,10))

for n,x in zip(range(60000),training_lable):
    training_lables[n][x] = 1
training_lable = training_lables

for x,i in zip(training_image,range(60000)):
    training_image[i] = I(x)
    
test_image = training_image[50000:]
test_lable = training_lable[50000:]
training_image = training_image[:50000]
training_lable = training_lable[:50000]
training_data = zip(training_image,training_lable)
test_data = zip(test_image,test_lable)
training_data = list(training_data)
test_data = list(test_data)





In [2]:
def sigmoid(z):#S型神经元
    return 1.0/(1.0+np.exp(-z))
print(training_image[0])
print(test_image[0])


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0
 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 1 1 1 1 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [2]:
def sigmoid(z):#S型神经元
    return 1.0/(1.0+np.exp(-z))
def sigmoid_prime(z):#S型神经元的导数：f‘(x)=f(x)*[1-f(x)]
    return (sigmoid(z) * (1-sigmoid(z)))
def relu(z):
    return np.maximum(z,0)
class Network(object):
    def __init__(self,sizes):
        self.sizes = sizes
        self.num_layers = len(sizes) #sipzes神经网络规模例如输入[10,20,10]就代表第一层10个神经元，第二层20个神经元，第三层10个神经元
        np.random.seed(0) #设置随机数种子，用于固定每次运行的随机参数，方便测试，可以略过此行代码
        self.biases = [np.random.randn(1,y) for y in sizes[1:]] #每层的偏移
        self.weights = [np.random.randn(x, y)                    #参数矩阵
                       for x, y in zip(sizes[:-1],sizes[1:])]
        print(self.weights[0].shape,self.weights[-1].shape)
    def Print(self):  #测试用
        print("SIZE:",self.sizes,)
        print("num_layers",self.num_layers)
#         print("B",self.biases[0],"W",self.weights[0].shape)
        print("B",len(self.biases),"W",len(self.weights))
    def test(self, test_data):   #验证函数
#         for x, y in test_data:
#             print(self.feedforward(x))
#             print(y)
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for x, y in test_data]
        return sum(y[int(x)] == 1 for (x, y) in test_results)
    def feedforward(self, a):       #前项计算
#         x = a.reshape(28,28)
#         fig = plt.figure()
#         plotwindow = fig.add_subplot(111)
#         plt.imshow(x , cmap='gray')
#         plt.show()
        a =  relu(np.dot(a, self.weights[0])+self.biases[0])
#         print("第一轮A：",a)
        a =  sigmoid(np.dot(a, self.weights[1])+self.biases[1])
#         print("第二轮A",a)
        return a
    def run(self,training_data,epochs,mini_batch_size,eta,test_data = None):
#         random.shuffle(training_data)  # 打乱测试数据
        if test_data: 
            test_data = list(test_data)
#             random.shuffle(test_data)
            test_data = test_data[:1000]
            n_test = len(test_data)
        training_data = list(training_data)
#         random.shuffle(training_data)
        # 测试版本
#         training_data = training_data[:100] #可以改变训练集大小，测试使用
        n = len(training_data)
        i = 0 #计数
        for j in range(epochs):
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            k = len(mini_batches)
            for  mini_batch in mini_batches:
                loss = self.training(mini_batch,mini_batch_size,eta)
            print('LOSS {0},'.format(loss))
            if j >= 0:
                print('Epoch {0}:{1} / {2}'. format(
                        j, self.test(test_data), n_test))
                
                ##训练函数，计算前向传播和反向传播，输出loss，并输出两项偏导数
                ##更新偏导数
    def Forward_and_BackForward(self, x, y):
                #Forward pass
        z = [] #中间变量矩阵
        z_s = [] #经过S型激活后的中间变量矩阵
        z_s.append(x)
#         print(np.dot(x,self.weights[0]).shape,self.biases[0].shape)
        a = np.dot(z_s[0],self.weights[0])+self.biases[0] #经过第一组参数矩阵
#         pdb.set_trace()
        z.append(a)
        z_s.append(relu(a))
        a = np.dot(z_s[1],self.weights[1]) + self.biases[1] #经过第二组参数矩阵
        z.append(a)    
#         print(a)
#         pdb.set_trace()
        z_s.append(sigmoid(a))
        y_pred = z_s[2]    
        #Loss ,计算loss并输出
        loss = np.sum(y_pred - y)
#         10*784 w1(784*30) w2(30*10)   #前项计算的具体过程，看着这个可以用来推反向传播
#         10*784 * w1(784*30) + B = z[0](10*30)
#         z_s[1] = R(z[0])
#         z_s[1](10*30) * w2(30*10)+  B = z[1](10*10)
#         z_s[2] = S(z[1])(10*10)
#         z_s[2](10*10)
        #Back Forward pass 反向传播
        
        grad_b = [np.zeros(b.shape) for b in self.biases]
        grad_w = [np.zeros(w.shape) for w in self.weights]
        grad_z = []
        
        a = (y_pred - y) * sigmoid_prime(z[-1]) #复合函数求导法,a是偏导数
        
        grad_z.append(a)
        grad_b[-1] = np.mean(a, axis = 0)
        grad_w[-1] = np.dot(z_s[-2].T,grad_z[0])
        a = np.dot(grad_z[0],self.weights[-1].T) #复合函数求导法
        a[z[0]<0] = 0
        grad_z.append(a)
        grad_b[-2] = np.mean(a, axis = 0)
        grad_w[-2] = np.dot(z_s[-3].T,grad_z[1])
        
#         for x in grad_w[-2]:
#             print(x)
#         pdb.set_trace()
        
        return (grad_w,grad_b,loss)
    def training(self, training_data, size,eta):
        
        #数据预处理，把成组的数据分开
        a = np.zeros((size,784))
        b = np.zeros((size,10))
        for (x, y), i in zip(training_data, range(size)):
            a[i] = x #转换成二维的矩阵，后面做乘法方便
            b[i] = y
        n_grad_w,n_grad_b,loss = self.Forward_and_BackForward(a,b)
        
        self.weights = [w-(eta/size)*nw
                        for w, nw in zip(self.weights, n_grad_w)]
        self.biases = [b-(eta/size)*nb
                        for b, nb in zip(self.biases, n_grad_b)]
        
        return loss

In [4]:
net = Network([784,30,10])
net.run(training_data,30,5,0.001,test_data)

(784, 30) (30, 10)
LOSS -1.999970155668049,
Epoch 0:146 / 1000
LOSS -3.99843915550367,
Epoch 1:142 / 1000
LOSS -4.0000006109738875,
Epoch 2:149 / 1000
LOSS -4.000086604201576,
Epoch 3:152 / 1000
LOSS -4.002829142765454,
Epoch 4:151 / 1000
LOSS -4.061907732621762,
Epoch 5:149 / 1000
LOSS -4.560263133804075,
Epoch 6:149 / 1000


KeyboardInterrupt: 

In [3]:
def sigmoid(z):#S型神经元
    return 1.0/(1.0+np.exp(-z))
def sigmoid_prime(z):#S型神经元的导数：f‘(x)=f(x)*[1-f(x)]
    return sigmoid(z) * (1-sigmoid(z))
class Network(object):
    def __init__(self,sizes):
        self.sizes = sizes
        self.num_layers = len(sizes) #sipzes神经网络规模例如输入[10,20,10]就代表第一层10个神经元，第二层20个神经元，第三层10个神经元
        np.random.seed(0) #设置随机数种子，用于固定每次运行的随机参数，方便测试，可以略过此行代码
        self.biases = [np.random.randn(1,y) for y in sizes[1:]] #每层的偏移
        self.weights = [np.random.randn(x, y)                    #参数矩阵
                       for x, y in zip(sizes[:-1],sizes[1:])]
        print(self.weights[0].shape,self.weights[-1].shape)
    def Print(self):  #测试用
        print("SIZE:",self.sizes,)
        print("num_layers",self.num_layers)
#         print("B",self.biases[0],"W",self.weights[0].shape)
        print("B",len(self.biases),"W",len(self.weights))
    def test(self, test_data):   #验证函数
#         for x, y in test_data:
#             print(self.feedforward(x))
#             print(y)
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for x, y in test_data]
        return sum(y[int(x)] == 1 for (x, y) in test_results)
    def feedforward(self, a):       #前项计算
#         x = a.reshape(28,28)
#         fig = plt.figure()
#         plotwindow = fig.add_subplot(111)
#         plt.imshow(x , cmap='gray')
#         plt.show()
        a =  sigmoid(np.dot(a, self.weights[0])+self.biases[0])
#         print("第一轮A：",a)
        a =  sigmoid(np.dot(a, self.weights[1])+self.biases[1])
#         print("第二轮A",a)
        return a
    def run(self,training_data,epochs,mini_batch_size,eta,test_data = None):
#         random.shuffle(training_data)  # 打乱测试数据
        if test_data: 
            test_data = list(test_data)
#             random.shuffle(test_data)
            test_data = test_data[:1000]
            n_test = len(test_data)
        training_data = list(training_data)
#         random.shuffle(training_data)
        # 测试版本
#         training_data = training_data[:100] #可以改变训练集大小，测试使用
        n = len(training_data)
        i = 0 #计数
        for j in range(epochs):
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            k = len(mini_batches)
            for  mini_batch in mini_batches:
                loss = self.training(mini_batch,mini_batch_size,eta)
            print('LOSS {0},'.format(loss))
            if j >= 0:
                print('Epoch {0}:{1} / {2}'. format(
                        j, self.test(test_data), n_test))
                
                ##训练函数，计算前向传播和反向传播，输出loss，并输出两项偏导数
                ##更新偏导数
    def Forward_and_BackForward(self, x, y):
                #Forward pass
        z = [] #中间变量矩阵
        z_s = [] #经过S型激活后的中间变量矩阵
        z_s.append(x)
#         print(np.dot(x,self.weights[0]).shape,self.biases[0].shape)
        a = np.dot(z_s[0],self.weights[0])+self.biases[0] #经过第一组参数矩阵
#         pdb.set_trace()
        z.append(a)
        z_s.append(sigmoid(a))
        a = np.dot(z_s[1],self.weights[1]) + self.biases[1] #经过第二组参数矩阵
        z.append(a)
#         print(a)
#         pdb.set_trace()
        z_s.append(sigmoid(a))
        y_pred = z_s[2]
        #Loss ,计算loss并输出
        loss = np.sum(y_pred - y)
#         10*784 w1(784*30) w2(30*10)   #前项计算的具体过程，看着这个可以用来推反向传播
#         10*784 * w1(784*30) + B = z[0](10*30)
#         z_s[1] = S(z[0])
#         z_s[1](10*30) * w2(30*10)+  B = z[1](10*10)
#         z_s[2] = S(z[1])(10*10)
#         z_s[2](10*10)
        #Back Forward pass 反向传播
        
        grad_b = [np.zeros(b.shape) for b in self.biases]
        grad_w = [np.zeros(w.shape) for w in self.weights]
        grad_z = []
        a = (y_pred - y) * sigmoid_prime(z[-1]) #复合函数求导法,a是偏导数
        grad_z.append(a)
        grad_b[-1] = np.mean(a, axis = 0)
        grad_w[-1] = np.dot(z_s[-2].T,grad_z[0])
        a = np.dot(grad_z[0],self.weights[-1].T)*sigmoid_prime(z[-2]) #复合函数求导法
#         print(a)
#         a[z[0]<0] = 0    #尝试加入relu层的反向传播，发现会导致隐藏层参数全部为零，所以没有加入。
#         print(a)
#         pdb.set_trace()
        
        grad_z.append(a)
        
        grad_b[-2] = np.mean(a, axis = 0)
        grad_w[-2] = np.dot(z_s[-3].T,grad_z[1])
        
        return (grad_w,grad_b,loss)
    def training(self, training_data, size,eta):
        
        #数据预处理，把成组的数据分开
        a = np.zeros((size,784))
        b = np.zeros((size,10))
        for (x, y), i in zip(training_data, range(size)):
            a[i] = x #转换成二维的矩阵，后面做乘法方便
            b[i] = y
        n_grad_w,n_grad_b,loss = self.Forward_and_BackForward(a,b)
        
        self.weights = [w-(eta/size)*nw
                        for w, nw in zip(self.weights, n_grad_w)]
        self.biases = [b-(eta/size)*nb
                        for b, nb in zip(self.biases, n_grad_b)]
        
        return loss


In [5]:
net = Network([784,30,10])
net.run(training_data,50,10,0.1,test_data)


(784, 30) (30, 10)
LOSS -3.969378961956361,
Epoch 0:484 / 1000
LOSS -0.971038649373509,
Epoch 1:610 / 1000
LOSS -0.8532505881912995,
Epoch 2:666 / 1000
LOSS -1.1291202241379805,
Epoch 3:740 / 1000
LOSS -1.4441089034829846,
Epoch 4:794 / 1000
LOSS -1.5063822784094518,
Epoch 5:817 / 1000
LOSS -1.678602093964041,
Epoch 6:832 / 1000
LOSS -1.7745329101493608,
Epoch 7:838 / 1000
LOSS -1.7140881473153775,
Epoch 8:841 / 1000
LOSS -1.5709149094579333,
Epoch 9:851 / 1000
LOSS -1.4449530885949025,
Epoch 10:853 / 1000
LOSS -1.3233974761309013,
Epoch 11:851 / 1000
LOSS -1.2417970220737977,
Epoch 12:858 / 1000
LOSS -1.1952911511944804,
Epoch 13:860 / 1000
LOSS -1.1637331566353823,
Epoch 14:862 / 1000
LOSS -1.1287499397272978,
Epoch 15:866 / 1000
LOSS -1.0832506429025452,
Epoch 16:869 / 1000
LOSS -1.0359971873712917,
Epoch 17:872 / 1000
LOSS -0.9949472269962866,
Epoch 18:875 / 1000
LOSS -0.9609194562134736,
Epoch 19:875 / 1000
LOSS -0.9317836594361208,
Epoch 20:877 / 1000
LOSS -0.904305512651482,
Epo

In [56]:
import pdb
# print(training_image.shape,training_lable.shape)
# print(test_image.shape,test_lable.shape)
# x = np.array([
#     [1],
#     [2],
#     [3],
# ])
# y = np.array([
#     [1],
#     [2],
#     [3],
# ])
mini_batches = [
    training_data[k:k+10]
    for k in range(0, 50000, 10)]

a = np.zeros((10,784))
b = np.zeros((10,10))
for  mini_batch in mini_batches:
    i = 0
    for x, y in mini_batch:
        a[i] = x
        b[i] = y
        i = i+1
    print(i)

# print(training_lable)     
# n = len(training_data)
# for k in range(0, n, 10):
#     print(k)
# mini_batches = [
#                 training_data[k:k+10]
#                 for k in range(0, n, 10)]
# x = mini_batches[3][2][0]
# x = x.reshape(28,28)
# fig = plt.figure()
# plotwindow = fig.add_subplot(111)
# plt.imshow(x , cmap='gray')
# plt.show()


10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
1

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
1

In [29]:
def sigmoid(z):#S型神经元
    return 1.0/(1.0+np.exp(-z))
def sigmoid_prime(z):#S型神经元的导数：f‘(x)=f(x)*[1-f(x)]
    return sigmoid(z) * (1-sigmoid(z))
class Network(object):
    def __init__(self,sizes):
        self.sizes = sizes
        self.num_layers = len(sizes) #sipzes神经网络规模例如输入[10,20,10]就代表第一层10个神经元，第二层20个神经元，第三层10个神经元
        np.random.seed(0) #设置随机数种子，用于固定每次运行的随机参数，方便测试，可以略过此行代码
        self.biases = [np.random.randn(1,y) for y in sizes[1:]] #每层的偏移
        self.weights = [np.random.randn(x, y)                    #参数矩阵
                       for x, y in zip(sizes[:-1],sizes[1:])]
        print(self.weights[0].shape,self.weights[-1].shape)
    def Print(self):  #测试用
        print("SIZE:",self.sizes,)
        print("num_layers",self.num_layers)
#         print("B",self.biases[0],"W",self.weights[0].shape)
        print("B",len(self.biases),"W",len(self.weights))
    def test(self, test_data):   #验证函数
#         for x, y in test_data:
#             print(self.feedforward(x))
#             print(y)
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for x, y in test_data]
        return sum(y[int(x)] == 1 for (x, y) in test_results)
    def feedforward(self, a):       #前项计算
#         x = a.reshape(28,28)
#         fig = plt.figure()
#         plotwindow = fig.add_subplot(111)
#         plt.imshow(x , cmap='gray')
#         plt.show()
        a =  np.dot(a, self.weights[0])+self.biases[0]
#         print("第一轮A：",a)
        a = np.maximum(a,0)  #relu层
        a = sigmoid(a)
        a =  sigmoid(np.dot(a, self.weights[1])+self.biases[1])
#         print("第二轮A",a)
        return a
    def run(self,training_data,epochs,mini_batch_size,eta,test_data = None):
#         random.shuffle(training_data)  # 打乱测试数据
        if test_data: 
            test_data = list(test_data)
#             random.shuffle(test_data)
            test_data = test_data[:1000]
            n_test = len(test_data)
        training_data = list(training_data)
#         random.shuffle(training_data)
        # 测试版本
#         training_data = training_data[:10000] #可以改变训练集大小，测试使用
        n = len(training_data)
        i = 0 #计数
        for j in range(epochs):
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            k = len(mini_batches)
            for  mini_batch in mini_batches:
                loss = self.training(mini_batch,mini_batch_size,eta)
            print('LOSS {0},'.format(loss))
            if j >= 0:
                print('Epoch {0}:{1} / {2}'. format(
                        j, self.test(test_data), n_test))
                
                ##训练函数，计算前向传播和反向传播，输出loss，并输出两项偏导数
                ##更新偏导数
    def Forward_and_BackForward(self, x, y):
                #Forward pass
        z = [] #中间变量矩阵
        z_s = [] #经过S型激活后的中间变量矩阵
        z_s.append(x)
#         print(np.dot(x,self.weights[0]).shape,self.biases[0].shape)
        a = np.dot(z_s[0],self.weights[0])+self.biases[0] #经过第一组参数矩阵
#         pdb.set_trace()
        z.append(a)
        a = np.maximum(a,0)   #测试用 relu层,可以略过此行代码
        z_s.append(sigmoid(a))
        a = np.dot(z_s[1],self.weights[1]) + self.biases[1] #经过第二组参数矩阵
        z.append(a)
#         print(a)
#         pdb.set_trace()
        z_s.append(sigmoid(a))
        y_pred = z_s[2]
        #Loss ,计算loss并输出
        loss = np.sum(y_pred - y)
#         1*784 w1(784*30) w2(30*10)   #前项计算的具体过程，看着这个可以用来推反向传播
#         1*784 * w1(784*30) = z[0]
#         z_s[1] = S(z[0])
#         relu_z_s[1](1*30) * w2(30*10) = z[1]
#         z_s[2] = S(z[1])
#         z_s[2](1*10)
        #Back Forward pass 反向传播
        grad_b = [np.zeros(b.shape) for b in self.biases]
        grad_w = [np.zeros(w.shape) for w in self.weights]
        grad_z = []
        a = (y_pred - y) * sigmoid_prime(z[-1]) #复合函数求导法,a是偏导数
        grad_z.append(a)
        grad_b[-1] = a
        grad_w[-1] = np.dot(z_s[-2].T,grad_z[0])
        a = np.dot(grad_z[0],self.weights[-1].T)*sigmoid_prime(z[-2]) #复合函数求导法
#         print(a)
#         a[z[0]<0] = 0    #尝试加入relu层的反向传播，发现会导致隐藏层参数全部为零，所以没有加入。
#         print(a)
#         pdb.set_trace()
        grad_z.append(a)
        grad_b[-2] = a
        grad_w[-2] = np.dot(z_s[-3].T,grad_z[1])
        
        return (grad_w,grad_b,loss)
    def training(self, training_data, size,eta):
        
        grad_b = [np.zeros(b.shape) for b in self.biases]
        grad_w = [np.zeros(w.shape) for w in self.weights]
        a = np.zeros((1,784))
        b = np.zeros((1,10))
        #数据预处理，把成组的数据分开
        for x, y in training_data:
            a[0] = x #转换成二维的矩阵，后面做乘法方便
            b[0] = y
            n_grad_w,n_grad_b,loss = self.Forward_and_BackForward(a,b)
            grad_b = [b+nb for b, nb in zip(grad_b,n_grad_b)]
            grad_w = [w+nw for w, nw in zip(grad_w,n_grad_w)]
        self.weights = [w-(eta/5000)*nw
                        for w, nw in zip(self.weights, grad_w)]
        self.biases = [b-(eta/5000)*nb
                        for b, nb in zip(self.biases, grad_b)]
        
        return loss


In [3]:
net = Network([784,30,10])
net.run(training_data,30,10,10,test_data)

(784, 30) (30, 10)


H:\anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


LOSS -0.5759617548194721,
Epoch 0:207 / 1000
LOSS -0.37898175399270445,
Epoch 1:299 / 1000
LOSS -0.4509613717909203,
Epoch 2:363 / 1000
LOSS -0.5132721330579109,
Epoch 3:403 / 1000
LOSS -0.21118208924143675,
Epoch 4:448 / 1000


KeyboardInterrupt: 

In [ ]:
def sigmoid(z):#S型神经元
    return 1.0/(1.0+np.exp(-z))
a = np.array([[1,2,3],[1,2,3]])
print(sigmoid(a))

数据
-0.16476648964589674

0.35530504324798495

In [69]:
training_data = zip(training_image,training_lable)
test_data = zip(test_image,test_lable)
training_data = list(training_data)
test_data = list(test_data)
random.shuffle(training_data)
random.shuffle(test_data)
print(len(training_data))
# print(training_data[0])
training_data = training_data[:100]
print(len(training_data))

50000
100


In [7]:


random.shuffle(training_data)
random.shuffle(test_data)
training_data = training_data[100:0]
#  mini_batches = [
#                 training_data[k:k+mini_batch_size]
#                 for k in xrange(0, n, 100)]

# print(len(training_data))
# n = len(training_data)
# mini_batches = [
#             training_data[k:100]
#             for k in range(0, n, 100)]
# print(len(mini_batches))
# print(mini_batches[0][0][0].shape)
# cows,cols,x = 100,784,10
# a = np.zeros((cows,cols))
# b = np.zeros((cows,x))
# for (x, y), i in zip(mini_batches[0],range(100)):
#     a[i] = x
#     b[i] = y
    
# print(a.shape)
# print(b[0])
# x = a[0].reshape(28,28)
# print(a[0].shape)
# fig = plt.figure()
# plotwindow = fig.add_subplot(111)
# plt.imshow(x , cmap='gray')
# plt.show()

# print(len(training_data))

# net = Network([784,30,10])
# net.Print()

50000
500
(784,)
(100, 784)
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
